In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import classy
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
# Function to compute the matter power spectrum using CLASS
def get_matter_power_spectrum_class(Omega_m, Omega_b, H0, ns, sigma8):
    # Define CLASS input parameters
    params = {
        'output': 'mPk',  # Request the matter power spectrum
        'P_k_max_1/Mpc': 1.0,  # Maximum k for the power spectrum
        'z_pk': '0.0',  # Compute power spectrum at redshift 0
        'h': H0 / 100,  # Dimensionless Hubble parameter
        'Omega_b': Omega_b,  # Baryon density parameter
        'Omega_cdm': Omega_m - Omega_b,  # CDM density parameter
        'n_s': ns,  # Scalar spectral index
        'sigma8': sigma8  # Normalization of the power spectrum
    }

    h = H0/100
    # Create an instance of the CLASS object
    cosmo = classy.Class()
    cosmo.set(params)
    cosmo.compute()
    
    # Get the matter power spectrum at z=0
    kh = np.logspace(-3, 0, 100)  # k values in h/Mpc
    pk = np.array([cosmo.pk(k*h, 0) for k in kh])  # Power spectrum values at z=0
    
    cosmo.struct_cleanup()
    cosmo.empty()
    
    return kh, pk

In [ ]:
# Plotting function
def plot_power_spectrum_class(Omega_m, Omega_b, H0, ns, sigma8):
    kh, pk = get_matter_power_spectrum_class(Omega_m, Omega_b, H0, ns, sigma8)
    
    # Plot the power spectrum
    plt.figure(figsize=(8, 6))
    plt.loglog(kh, pk, label=r'$P(k)$', color='g', lw=1.5)
    plt.xlabel(r'Wave number $k$ [h/Mpc]')
    plt.ylabel(r'Power Spectrum $P(k)$')
    plt.title('Matter Power Spectrum from CLASS')
    plt.ylim(10**2,2.5*10**5)
    plt.xlim(10**(-3),1)
    plt.grid(True, which='both', ls='--', lw=0.5)
    plt.legend()
    plt.show()

In [ ]:
# Create interactive widgets
interact(
    plot_power_spectrum_class,
    Omega_m=widgets.FloatSlider(min=0.1, max=0.5, step=0.01, value=0.3, description=r'$\Omega_m$'),
    H0=widgets.FloatSlider(min=50, max=80, step=1, value=70, description=r'H$_0$'),
    ns=widgets.FloatSlider(min=0.9, max=1.1, step=0.01, value=0.96, description=r'$n_s$'),
    sigma8=widgets.FloatSlider(min=0.6, max=1.0, step=0.01, value=0.8, description=r'$\sigma_8$'),
    Omega_b=widgets.FloatSlider(min=0.02, max=0.08, step=0.005, value=0.05, description=r'$\Omega_b$')
)